In [21]:
# Import the necessary libraries
import numpy as np
from scipy.integrate import quad

## Lewis (2001) Approach
We already know, the value of a call option under Lewis (2001) is determine by:
$$\begin{equation*} C_0 = S_0 - \frac{\sqrt{S_0K}e^{-rT}}{\pi} \end{equation*} \int_0^\infty \mathbf{Re}[e^{izk} \varphi^{B96}(z - i/2)] \frac{dz}{z^2 + 1/4}$$

Where $\varphi^{B96}()$ is the characteristics function of the model. In this case, the characteristics function of Bates (1996) was given by

$$\varphi_0^{B96}(u, T) = \varphi_0^{H93}\varphi_0^{M76J}(u, T)$$
which is enssentially the product of two characteristis functions. Let's first define each of these characteristics functions.

### Characteristics Functions
#### Heston (1993) Characteristics Function
The characteristics function of Heston (1993) is given by
$$\varphi{H93}(u, T) = e^{H_1(u, T) + H_2(u, T)v_o}$$
where 
$$
\
\begin{equation*}
H_1 (u, T) \equiv r_0 uiT + \frac{c_1}{\sigma_\nu^2}\Biggl\{(\kappa_\nu - \rho \sigma_\nu ui+c_2) T - 2 log \left[\frac{1 - c_3e^{c_2T}}{1 - c_3}\right]  \Biggl\}
\end{equation*}
$$
$$\begin{equation*} H_2(u, T) = \frac{\kappa_\nu - \rho\sigma_\nu ui + c_2}{\sigma_\nu} \left[\frac{1 - e^{c_2T}}{1 - c_3e^{c_2T}}\right] \end{equation*}$$

$$c_1 = \kappa_\nu \theta_\nu$$
$$c2 = - \sqrt{(\rho\sigma_\nu ui - \kappa_\nu)^2 - \sigma_\nu^2(-ui - u^2)} $$

$$c3 \equiv \frac{\kappa_\nu - \rho \sigma_\nu ui + c_2}{\kappa_\nu - \rho \sigma_\nu ui - c_2}

In [22]:
def H93_char_fun(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """ Valuation of European call in H93 model via Lewis (2001) Fourier-based apprach:
     characteristic function.
      Parameter definition see function BCC_call_value """
    c1 = kappa_v * theta_v
    c2 = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v)**2 - sigma_v**2 * (-u *1j - u**2)
        )
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2)/(
        kappa_v - rho * sigma_v * u * 1j - c2)

    H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2*T))/(1 - c3))
        )
    H2 = (
        (kappa_v - rho * sigma_v * u * 1j + c2)
        / sigma_v**2 *
        ((1 - np.exp(c2 *T))/(1 - c3 * np.exp(c2*T)))
    )
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

### Merton (1976) Adjusted Characyteristics Function (Only Jump Component)
In order to produce a stocahstic volatity jump-difusion model, we need to incorporate Only the jump component of the Merton (1976) characteristic function.

The adjusted (only Jump) characteristics function of Merton (1976) is therefore given by:
$$\begin{equation*} \varphi_0^{M76J}(u, T) = e^{\left(\left( iu\omega + \lambda(e^{iu \mu_j -u^2\delta^2/2} - 1)\right)T\right)} \end{equation*}$$

where, 
$$\omega = -\lambda \left( e^{\mu_j + \delta^2/2} - 1 \right)$$

Let's Code


In [23]:
def M76J_char_func(u, T, lamb, mu, delta):
    """
    Adjusted Characteristic function for  Merton '76 Model: Only jump component
    """

    omega = -lamb * (np.exp(mu + 0.5 *delta**2 ) - 1)
    char_func_value = np.exp(
        (1j * u * omega + lamb * (np.exp(1j * u * mu - u**2 * 0.5 * delta**2) - 1) * T)
    )

    return char_func_value

### Bates (1996) Characteristic Function
Now we can combine both previous characteristics functions to produce the one we are interested in

In [24]:
def B96_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta):
    """
    Bates (1996) characteristic function
    """
    H93 = H93_char_fun(u, T, r, kappa_v, theta_v, sigma_v, rho, v0)
    M76J = M76J_char_func(u, T, lamb, mu, delta)

    return H93 * M76J

### Call and Integral Value in Bates (1996)
The next step is to calculate the value of the Lewis (2001) integral for the specific case of the Bates (1996) characteristic function

In [25]:
def B96_int_func(u, S0, K, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta):
    """
    Lewis (2001) integral value for Bates 1996 characteristic function.
    """
    char_func_value = B96_char_func(
        u -1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
    )
    int_func_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    )
    return int_func_value

And, finally, we also need a function to compute the overall call option value once we have all the ingredients

In [26]:
def B96_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta):
    """
    Valuation of European call option in B96 Model via Lewis (2001)
    Parameters:
    =============
    S0:         float
                initial stock/index level
    K:          float
                strike price
    T:          float
                Time to maturity (for t=0)
    r:          float
                constant risk-free short rate
    kappa_v:    float
                mean-reversion factor
    theta_v:    float
                long-run mean of variance
    sigma_v:    float
                volatility of variance
    rho:        float
                corelation between variance and stock/index level
    v0:         float
                initial level of variance
    lamb:       float
                jump intensity
    mu:         float
                expected jump size
    delta:      float
                standard deviation of jump
    =============
    """
    int_value = quad(lambda u: B96_int_func(
        u, S0, K, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
    ),
    0,
    np.inf,
    limit=250
    )[0]
    call_value = max(0, S0 - (np.sqrt(S0 * K) * np.exp(-r * T))/np.pi * int_value)
    return call_value

### Pricing via Lewis (2001) with bates (1996)

In [27]:
# General Parameter
S0 = 100
K = 100
T = 1
r = 0.05

# Heston '93 parameters
kappa_v = 1.5
theta_v = 0.02
sigma_v = 0.15
rho = 0.1
v0 = 0.01

# merton '76 parameters
lamb = 0.25
mu = -0.2
delta = 0.1
sigma = np.sqrt(v0)

In [28]:
print(
    "B96 Call option price via Lewis(2001): $%10.4f" % B96_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
)

B96 Call option price via Lewis(2001): $    8.9047


### FFT Approach - Carr and Madan (1999)
As an alternative to Lewis (2001), we could also implement the FFT algorithim. essentially, we can aply FFT to the integral in the call option price derived by Carr and Madan (1999):
$$\begin{equation*} C_0 = \frac{e^{-\alpha k}}{\pi} \int_0^\infty  e^{i\nu k}\frac{e^{-rT} \varphi^{B96}(\nu - (\alpha + 1)i, T)}{\alpha^2 + \alpha - \nu^2 + i(2\alpha + 1)\nu} d\nu \end{equation*}$$

In [49]:
def B96_call_FFT(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta):
    """
    Call option price in Bates (1996) under FFT
    """
    k = np.log(K / S0)
    g = 1 # factor to increase accuracy
    N = g * 4096
    eps = (g * 150) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    vo = eta * (u - 1)

    # Modifications to ensure integrability
    if S0 >= 0.95 * K: # ITM Case
        alpha = 1.5
        v = vo - (alpha + 1) * 1j
        modcharFunc = np.exp(-r * T) * (
            B96_char_func(v, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
            / (alpha**2 + alpha - vo**2 + 1j * (2 * alpha + 1) * vo)
        )
        # print(modcharFunc)
    else:
        alpha = 1.1
        v = (vo - 1j * alpha) - 1j
        modcharFunc1 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo - 1j * alpha ))
            - np.exp(r * T) / (1j * (vo - 1j * alpha))
            - B96_char_func(
                v, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
            )
            / ((vo - 1j * alpha) ** 2 - 1j * (vo + 1j * alpha))
        )
        # print(modcharFunc1)

        v = (vo + 1j * alpha) - 1j

        modcharFunc2 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo + 1j * alpha))
            - np.exp(r * T) / (1j * (vo + 1j * alpha))
            - B96_char_func(
                v, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta
            )
            / ((vo + 1j * alpha) **2 - 1j * (vo + 1j * alpha))
        )
        # print(modcharFunc2)

    # Numerical FFT Routine
    delt = np.zeros(N)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    simpsonW = (3 + (-1) ** j - delt) / 3
    if S0 >= 0.95 * K:
        FFTFunc = np.exp(1j * b * vo) * modcharFunc * eta * simpsonW
        payoff = (np.fft.fft(FFTFunc)).real
        CallValueM = np.exp(-alpha * k) / np.pi * payoff
    else:
        FFTFunc = (
            np.exp(1j * b * vo) * (modcharFunc1 - modcharFunc2) * 0.5 * eta * simpsonW
        )
        payoff = (np.fft.fft(FFTFunc)).real
        CallValueM = payoff / (np.sinh(alpha * k) * np.pi)

    pos = int((k + b) / eps)
    CallValue = CallValueM[pos] * S0

    return CallValue

In [50]:
print(
    "B96 Call option price via FFT: $%10.4f"
    % B96_call_FFT(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)
)

B96 Call option price via FFT: $    8.9047
